# 線形計画法

- シンプレックス法

In [40]:
# display graph on notebook
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

In [141]:
"""
最大化する目的関数
z = 2x1 + 3x2

制約条件
x1 + 2x2 <= 14
x1 + x2 <= 8
3x1 + x2 <= 18

解
x1 = 2
x2 = 6
z = 22
"""
objective1 = np.array([2., 3.])
constraints1 = np.array(
    [
        [1., 2., 14.],
        [1., 1., 8.],
        [3., 1., 18.]
    ]
)

"""
最大化する目的関数
z = 2x1 - x2

制約条件（右辺に負の数あり）
2x1 - x2 <= 2
x1 - 5x2 <= -4

解
x1 = 
x2 = 
z = 
"""
objective2 = np.array([2., -1.])
constraints2 = np.array(
    [
        [2., -1., 2],
        [1., -5., -4]
    ]
)

In [238]:
def simplex_lp(objective, constraints):
    """
    Parameters
    ----------
    objective : numpy.array
        最大化したい目的関数の係数ベクトル (n次元)
        c1 x1 + c2 x2 + ... + cn xn

    constraints : numpy.array
        制約条件の係数行列 (m行n+1列)
        a11 x1 + a12 x2 + ... + a1n xn <= b1
        ...
        am1 x1 + am2 x2 + ... + amn xn <= bm
    """
    # 処理しやすいように整形
    m = constraints.shape[0]
    n = constraints.shape[1] - 1
    a = constraints[:,0:n]
    b = constraints[:,-1]
    
    # 実行可能解を持つかどうかの判定
    # 負のbを持つ場合は補助問題を先に解く必要がある
    if np.any(b < 0):
        print('\n---------- 実行可能解の存在を調べる ----------\n')
        col_minus_slack = np.array([-1] * m)
        a_sub = np.concatenate((a, np.identity(m), np.reshape(col_minus_slack, (m, 1))), axis=1)
        b_sub = np.reshape(b, (m, 1))
        constraints_sub = np.concatenate((a_sub, b_sub), axis=1)
        objective_sub = np.concatenate(([0]*(n+m), [1, 0]), axis=0).reshape(1,n+m+2)
        matrix_sub = np.concatenate((constraints_sub, objective_sub), axis=0)
        print(matrix_sub)
        # マイナスの補正変数が必要だった列は基底にできるので掃き出しを行う
        i_b_minus = [i for i in range(m) if b[i] < 0][0]
        matrix_sub[i_b_minus] *= -1
        for i in range(m+1):
            if i != i_b_minus:
                matrix_sub[i] -= matrix_sub[i][n+m] * matrix_sub[i_b_minus]
        print(matrix_sub)
        # 補助問題の最適解を求め、ゼロかどうか調べる
        cycle(matrix_sub)
        if matrix_sub[-1][-1] != 0:
            raise Exception('実行可能解を持たない問題です')
        # 補助問題の結果を使い、元の問題の初期実行可能解に対応するスラック形を求める
        a_new = matrix_sub[:m,:n+m]
        b_new = np.reshape(matrix_sub[:m,-1], (m, 1))
        constraints_new = np.concatenate((a_new, b_new), axis=1)
        objective_new = np.concatenate((objective * -1, [0]*(m+1)), axis=0).reshape(1,n+m+1)
        matrix = np.concatenate((constraints_new, objective_new), axis=0)
        i_base_colmn = []
        for j in range(n+m):
            cnt_1 = 0
            cnt_0 = 0
            for i in range(m):
                if matrix[i][j] == 1:
                    cnt_1 += 1
                    i_1 = i
                elif matrix[i][j] == 0:
                    cnt_0 += 1
                else:
                    break
            if cnt_1 == 1 and cnt_0 == m-1:
                matrix[m] -= matrix[i_1] * matrix[m][j]
    else:
        constraints_new = np.concatenate((a, np.identity(m), np.reshape(b, (m, 1))), axis=1)
        objective_new = np.concatenate((objective * -1, [0]*(m+1)), axis=0).reshape(1,n+m+1)
        matrix = np.concatenate((constraints_new, objective_new), axis=0)

    print('\n---------- 最適解を求める ----------\n')
    print(matrix)
    cycle(matrix)


def cycle(matrix):
    try_num = 0
    try_num_max = 100
    while try_num < try_num_max:
        try_num += 1
    
        # 変数 x_i を増やせば目的関数が大きくなる係数 c_i を見つける
        i_c_min = 0
        c_min = matrix[-1][i_c_min]
        for i in range(matrix.shape[1]-1):
            if matrix[-1][i] < c_min:
                i_c_min = i
                c_min = matrix[-1][i_c_min]
        if c_min >= 0:
            # どの x_i を増やしても目的関数は増大しない状態なので処理終了
            break
    
        # x_i_c_min を増やしていったときに一番早く制約の上限/下限に達するものを見つける
        i_fastest_limit = 0
        fastest_limit = math.fabs(matrix[i_fastest_limit][-1] / matrix[i_fastest_limit][i_c_min])
        for i in range(matrix.shape[0]-1):
            limit = math.fabs(matrix[i][-1] / matrix[i][i_c_min])
            if limit < fastest_limit:
                i_fastest_limit = i
                fastest_limit = limit
    
        matrix[i_fastest_limit] /= matrix[i_fastest_limit][i_c_min]
        for i in range(matrix.shape[0]):
            if i != i_fastest_limit:
                matrix[i] -= matrix[i_fastest_limit] * matrix[i][i_c_min]

        print('\nTrial {}:'.format(try_num))
        print(matrix)
        print('objective = {}'.format(matrix[-1][-1]))

In [239]:
simplex_lp(objective1, constraints1)


---------- 最適解を求める ----------

[[ 1.  2.  1.  0.  0. 14.]
 [ 1.  1.  0.  1.  0.  8.]
 [ 3.  1.  0.  0.  1. 18.]
 [-2. -3.  0.  0.  0.  0.]]

Trial 1:
[[ 0.5  1.   0.5  0.   0.   7. ]
 [ 0.5  0.  -0.5  1.   0.   1. ]
 [ 2.5  0.  -0.5  0.   1.  11. ]
 [-0.5  0.   1.5  0.   0.  21. ]]
objective = 21.0

Trial 2:
[[ 0.  1.  1. -1.  0.  6.]
 [ 1.  0. -1.  2.  0.  2.]
 [ 0.  0.  2. -5.  1.  6.]
 [ 0.  0.  1.  1.  0. 22.]]
objective = 22.0


In [240]:
simplex_lp(objective2, constraints2)


---------- 実行可能解の存在を調べる ----------

[[ 2. -1.  1.  0. -1.  2.]
 [ 1. -5.  0.  1. -1. -4.]
 [ 0.  0.  0.  0.  1.  0.]]
[[ 1.  4.  1. -1.  0.  6.]
 [-1.  5. -0. -1.  1.  4.]
 [ 1. -5.  0.  1.  0. -4.]]

Trial 1:
[[ 1.8  0.   1.  -0.2 -0.8  2.8]
 [-0.2  1.  -0.  -0.2  0.2  0.8]
 [ 0.   0.   0.   0.   1.   0. ]]
objective = 0.0

---------- 最適解を求める ----------

[[ 1.8  0.   1.  -0.2  2.8]
 [-0.2  1.  -0.  -0.2  0.8]
 [-1.8  0.   0.   0.2 -0.8]]

Trial 1:
[[ 1.00000000e+00  0.00000000e+00  5.55555556e-01 -1.11111111e-01
   1.55555556e+00]
 [ 0.00000000e+00  1.00000000e+00  1.11111111e-01 -2.22222222e-01
   1.11111111e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  5.55111512e-17
   2.00000000e+00]]
objective = 1.9999999999999998
